In [ ]:
# SETUP ENV
import matplotlib.pyplot as plt
plt.switch_backend('tkagg')
from pycigar.utils.input_parser import input_parser
import numpy as np
from pycigar.utils.registry import register_devcon
import tensorflow as tf
from ray.rllib.models.catalog import ModelCatalog
from gym.spaces import Tuple, Discrete, Box
import matplotlib
import random
import re

# misc_inputs = '/home/toanngo/Documents/GitHub/cigar-document/ceds-cigar/pycigar/data/ieee37busdata_regulator_attack/misc_inputs.csv'
# dss = '/home/toanngo/Documents/GitHub/ceds-cigar/pycigar/data/ieee37busdata_regulator_attack/ieee37.dss'
# load_solar = '/home/toanngo/Documents/GitHub/cigar-document/ceds-cigar/pycigar/data/ieee37busdata_regulator_attack/load_solar_data.csv'
# breakpoints = '/home/toanngo/Documents/GitHub/cigar-document/ceds-cigar/pycigar/data/ieee37busdata_regulator_attack/breakpoints.csv'


misc_inputs = 'C:\\Users\\kathl\\Desktop\\Github\\ceds-cigar\\pycigar\\data\\ieee37busdata_regulator_attack\\misc_inputs.csv'
dss = 'C:\\Users\\kathl\\Desktop\\Github\\ceds-cigar\\pycigar\\data\\ieee37busdata_regulator_attack\\ieee37.dss'
load_solar = 'C:\\Users\\kathl\\Desktop\\Github\\ceds-cigar\\pycigar\\data\\ieee37busdata_regulator_attack\\load_solar_data.csv'
breakpoints = 'C:\\Users\\kathl\\Desktop\\Github\\ceds-cigar\\pycigar\\data\\ieee37busdata_regulator_attack\\breakpoints.csv'


start = 100
hack = 0.4
sim_params = input_parser(misc_inputs, dss, load_solar, breakpoints, benchmark=True, vectorized_mode=True, percentage_hack=hack)
sim_params['scenario_config']['start_end_time'] = [start, start + 750]
del sim_params['attack_randomization']
for node in sim_params['scenario_config']['nodes']:
    node['devices'][0]['adversary_controller'] =  'adaptive_unbalanced_fixed_controller'

from pycigar.envs import CentralControlPhaseSpecificPVInverterEnv
env = CentralControlPhaseSpecificPVInverterEnv(sim_params=sim_params)
env.reset()
done = False
while not done:
    _, r, done, _ = env.step([10, 10, 10])

In [ ]:
from pycigar.utils.logging import logger
log_dict = logger().log_dict
custom_metrics = logger().custom_metrics

In [ ]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.resources import INLINE
output_notebook(INLINE)

In [4]:
import pandas as pd
import networkx 
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure

from bokeh.models import CategoricalColorMapper, HoverTool, BoxSelectTool, ColumnDataSource, Panel, Column, Band, BoxAnnotation, Row, Ellipse, Legend
from bokeh.models.widgets import CheckboxGroup, CheckboxButtonGroup, Slider, RangeSlider, Tabs, MultiChoice

from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Category20_20 as palette

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
import itertools  


from bokeh.models import  CustomJS, StaticLayoutProvider, Oval, Circle, LabelSet
from bokeh.models import   GraphRenderer
from bokeh.models.widgets import  Button, DataTable, TableColumn, NumberFormatter
from bokeh.plotting import figure, from_networkx
from bokeh.models.graphs import  NodesAndLinkedEdges, EdgesAndLinkedNodes, NodesOnly


from bokeh.models import Range1d, Circle, MultiLine
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap

In [5]:
log_dict.keys()

dict_keys(['u_metrics', 'v_metrics', 'v_worst_metrics', 'y_metrics', 'component_observation', 'current', 'inverter_s701a', 'adversary_inverter_s701a', 'inverter_s701b', 'adversary_inverter_s701b', 'inverter_s701c', 'adversary_inverter_s701c', 'inverter_s712c', 'adversary_inverter_s712c', 'inverter_s713c', 'adversary_inverter_s713c', 'inverter_s714a', 'adversary_inverter_s714a', 'inverter_s714b', 'adversary_inverter_s714b', 'inverter_s718a', 'adversary_inverter_s718a', 'inverter_s720c', 'adversary_inverter_s720c', 'inverter_s722b', 'adversary_inverter_s722b', 'inverter_s722c', 'adversary_inverter_s722c', 'inverter_s724b', 'adversary_inverter_s724b', 'inverter_s725b', 'adversary_inverter_s725b', 'inverter_s727c', 'adversary_inverter_s727c', 'inverter_s728', 'adversary_inverter_s728', 'inverter_s729a', 'adversary_inverter_s729a', 'inverter_s730c', 'adversary_inverter_s730c', 'inverter_s731b', 'adversary_inverter_s731b', 'inverter_s732c', 'adversary_inverter_s732c', 'inverter_s733a', 'adve

In [6]:
inverter_list = []
adversary_inv = []
for i in log_dict.keys():
    if 'inverter_' in i and 'adversary' not in i:
        inverter_list.append(i)
    elif 'adversary' in i:
        adversary_inv.append(i)
print(inverter_list, "\n")
print(adversary_inv)

['inverter_s701a', 'inverter_s701b', 'inverter_s701c', 'inverter_s712c', 'inverter_s713c', 'inverter_s714a', 'inverter_s714b', 'inverter_s718a', 'inverter_s720c', 'inverter_s722b', 'inverter_s722c', 'inverter_s724b', 'inverter_s725b', 'inverter_s727c', 'inverter_s728', 'inverter_s729a', 'inverter_s730c', 'inverter_s731b', 'inverter_s732c', 'inverter_s733a', 'inverter_s734c', 'inverter_s735c', 'inverter_s736b', 'inverter_s737a', 'inverter_s738a', 'inverter_s740c', 'inverter_s741c', 'inverter_s742a', 'inverter_s742b', 'inverter_s744a'] 

['adversary_inverter_s701a', 'adversary_inverter_s701b', 'adversary_inverter_s701c', 'adversary_inverter_s712c', 'adversary_inverter_s713c', 'adversary_inverter_s714a', 'adversary_inverter_s714b', 'adversary_inverter_s718a', 'adversary_inverter_s720c', 'adversary_inverter_s722b', 'adversary_inverter_s722c', 'adversary_inverter_s724b', 'adversary_inverter_s725b', 'adversary_inverter_s727c', 'adversary_inverter_s728', 'adversary_inverter_s729a', 'adversary

---

### Network order

In [7]:
import opendssdirect as dss
dss_file = 'C:\\Users\\kathl\\Desktop\\Github\\ceds-cigar\\pycigar\\data\\ieee37busdata_regulator_attack\\ieee37.dss'
dss.run_command('Redirect ' + dss_file)

''

In [8]:
def generate_network_order():
    bus1 = []
    bus2 = []
    for line in range(len(dss.Lines.AllNames())):
        dss.Lines.Name(dss.Lines.AllNames()[line])   
        bus1.append((dss.Lines.Bus1().split('.')[0]))
        bus2.append((dss.Lines.Bus2().split('.')[0]))
    return bus1, bus2

bus1, bus2 = generate_network_order()

---

## Current App

In [61]:
def y_plot():
    
    node_end = []
    # list of all the nodes that inverters/adversary inverters operate at
    for i in inverter_list:
        temp = i.split('_')[1]
        node_end.append(temp)
    node_end.append('y_mean')
    node_end.append('y_worst')
    
    x = pd.DataFrame({'Source': bus1, 'Target':bus2, 'Weight':np.ones(len(bus2))})

    # store y data
    df = pd.DataFrame()
    for inv in inverter_list:
        df = df.append(pd.Series(log_dict[inv]['y'], name=inv))

    tags = ['y_worst_node', 'y_worst', 'y_mean']
    for tag in tags:
        df = df.append(pd.Series(data=log_dict['y_metrics'][tag], name=tag))
 
    
    y_df = df.transpose()

    # store adversary y data
    adv_y_df = pd.DataFrame()
    for inv in adversary_inv:
        adv_y_df = adv_y_df.append(pd.Series(log_dict[inv]['y'], name=inv))


    adv_y_df = adv_y_df.transpose()
    
    # combine y data
    all_y_df = pd.concat([y_df, adv_y_df], axis=1)

    imbalance_df = all_y_df*100
 
    
    c = itertools.cycle(palette)
    colors = {}
    for node in node_end:
        colors[node] = next(c)

    print(colors)
    def make_dataset(dataset, node_list, range_start=0, range_end=650):
        
             
        updated_dataset = pd.DataFrame()
        
        # select all nodes that are marked
        for node in node_list:
            
            updated_dataset = pd.concat([updated_dataset, dataset.loc[:, [node in s for s in dataset.columns.values]]], axis=1)
        
        # also select y worst and y mean
        updated_dataset = pd.concat([updated_dataset, dataset['y_worst']], axis=1)
        
        updated_dataset = pd.concat([updated_dataset, dataset['y_mean']], axis=1)
        
      

        return ColumnDataSource(updated_dataset.loc[int(range_start):int(range_end), :])

    def make_plot(src, nodes, a_d, title='Oscillation Intensity', x_axis_label='Time (s)', y_axis_label='Y Metric'):      

        p = figure(plot_width=800, plot_height=300,
                 title=title,x_axis_label=x_axis_label, y_axis_label=y_axis_label)

        for node in nodes:
            
            # would have to move this out if using network to represent the nodes
            if node == 'y_worst' or node == 'y_mean':
                    glyph = p.line(source=src, x='index', y=node, legend_label=node, color=colors[node], line_width=2)
                    hover = HoverTool(renderers=[glyph], tooltips=[('{}'.format(node), '$y')])
                    p.add_tools(hover)

            elif 0 in a_d: # attacker/adversary checked
                glyph = p.line(source=src, x='index', y="inverter_"+ node, legend_label=node, color=colors[node], line_width=2)
                hover = HoverTool(renderers=[glyph], tooltips=[('adversary_inverter_{}'.format(node), '$y')])
                p.add_tools(hover)
            elif 1 in a_d: # defender
                glyph = p.line(source=src, x='index', y="inverter_"+ node, legend_label=node, color=colors[node], line_width=2)
                hover = HoverTool(renderers=[glyph], tooltips=[('inverter_{}'.format(node), '$y')])
                p.add_tools(hover)
            
        return p
    
    u_df = pd.DataFrame(log_dict['u_metrics'])*100
    all_node_list = u_df.columns.to_list()

    u_df['u_std_lower'] = u_df['u_mean'] - u_df['u_std']
    u_df['u_std_upper'] = u_df['u_mean'] + u_df['u_std']
    
    ######################################

#     c = itertools.cycle(palette)
#     colors = {}
#     for node in all_node_list:
#         colors[node] = next(c)
    #print(colors)
    def make_u_dataset(dataset, nodes, range_start=0, range_end=650):
   
        
        nodes = [re.findall("\d+", i)[0] for i in nodes]
     
       
        if 'u_std' in nodes:
            nodes.append('u_std_lower')
            nodes.append('u_std_upper')
        return ColumnDataSource(dataset[nodes][int(range_start):int(range_end)])
    
    def make_u_plot(src, nodes, title='Oscillation Imbalance', x_axis_label='Time (s)', y_axis_label='U Metric'):
       
        clipped_node = [re.findall("\d+", i)[0] for i in nodes]
       
     
        p = figure(plot_width=800, plot_height=300, 
                   title=title,
                   x_range=(src.data['index'][0], src.data['index'][-1]), y_range=(0, 8),
                   x_axis_label=x_axis_label, y_axis_label=y_axis_label)
        for node in range(len(clipped_node)):
            if clipped_node[node] != 'u_std' and clipped_node[node] != 'u_std_lower' and clipped_node[node] != 'u_std_upper':

                glyph = p.line(source=src, x='index', y=clipped_node[node], legend_label=nodes[node], color=colors[nodes[node]], line_width=2)
                hover = HoverTool(renderers=[glyph], tooltips=[('imbalance_{}'.format(clipped_node[node]), '@{}'.format(clipped_node[node]))])#, mode='vline')
                p.add_tools(hover)
            elif node == 'u_std':               
                band = Band(source=src, base='index', lower='u_std_lower', upper='u_std_upper', 
                level='underlay', fill_alpha=0.5, line_width=0.5, fill_color='green')

                p.add_layout(band)
            else:
                pass

        high_box = BoxAnnotation(bottom=6, fill_alpha=0.05, fill_color='red')
        mid_box = BoxAnnotation(bottom=3, top=6, fill_alpha=0.05, fill_color='orange')
        p.add_layout(high_box)
        p.add_layout(mid_box)
        
        return p
    
    #######################################
    
    def make_tap_dataset(regname=['reg1']):
        
        x = []
        y = []
        
        for reg in regname:
            x.append(np.arange(len(log_dict[reg]['tap_number'])))
            y.append(log_dict[reg]['tap_number'])
            
        return x, y

    def make_tap_plot(x, y, regname, title='Tap Number Changes', x_axis_label='Time (s)', y_axis_label='Tap Number'):

        p = figure(plot_width=800, plot_height=300, 
                   title=title,
                   x_axis_label=x_axis_label, y_axis_label=y_axis_label)   

        for i in range(len(x)):

            glyph = p.line(x=x[i], y=y[i],legend_label=regname[i])
            hover = HoverTool(renderers=[glyph], tooltips=[("Tap Number", "$y")])
            p.add_tools(hover)

        return p
    
    ######################################

    def generate_voltage_df(phase):
        #phase - base-0

        v_df = pd.DataFrame()

        for val in log_dict['v_metrics'].values():

            for i in val:

                vals = np.array(list(i.values()))
                key = np.array(list(i.keys()))

                temp = pd.DataFrame(data=vals[:, phase], index=key)
                v_df = pd.concat([v_df, temp], axis = 1)

        v_a = v_df.transpose().reset_index().drop(labels='index', axis=1)

        return v_a  


    def make_voltage_dataset(nodes, select_stats):
        
        nodes = [re.findall("\d+", i)[0] for i in nodes]
        
        v_a = generate_voltage_df(0)
        v_b = generate_voltage_df(1)
        v_c = generate_voltage_df(2)

        all_voltages = [v_a, v_b, v_c]
        all_node_list = v_c.columns

        dataset = []
        for i in range(len(select_stats)):
            dataset.append(all_voltages[int(select_stats[i])].loc[:, nodes])           
        
        return dataset
    
    def make_voltage_plot(src, nodes, phases, title='Bus Voltage', x_axis_label='Time (s)', y_axis_label='Voltage (V)'):
        clipped_nodes = [re.findall("\d+", i)[0] for i in nodes]
        p = figure(plot_width=800, plot_height=300, 
                title=title,
                x_axis_label=x_axis_label, y_axis_label=y_axis_label)    


        p.xaxis.major_label_orientation = "vertical" # alternatively, math.pi/2

        color_dict = {0: 'A', 1:'B', 2:'C'}
        legend_it = []

        for s in range(len(phases)):
            for node in range(len(nodes)):
              
                subset = src[s].iloc[:, node]

                glyph = p.line(y=list(subset.values), x=list(subset.index), legend_label=nodes[node],color=colors[nodes[node]])  

                legend_it.append((nodes[node], [glyph]))

                hover = HoverTool(renderers=[glyph], tooltips=[('node {}'.format(nodes[node]), 'phase {}'.format(color_dict[phases[s]])),("Voltage", "$y")])
                p.add_tools(hover)

        return p, legend_it
    
    ######################################
    
    
    ##### Functions
    def get_control_params(k):
        x = np.array(log_dict[k]['control_setting'])
        y = pd.DataFrame(data=x, columns=['control1', 'control2', 'control3', 'control4', 'control5'])
        y = y.transpose()

        return y

    def get_inv_control_dict(inv_data):
        inv_control_dict = {}
        for k in inv_data:
            inv_control_dict[k] = get_control_params(k)
        return inv_control_dict

    def get_translation_and_slope(a_val, init_a):
        points = np.array(a_val)
        slope = points[:, 1] - points[:, 0]
        translation = points[:, 2] - init_a[2]
        return translation, slope

    def all_inv_translation_and_slope(inv_data):
        t_s_dict = {}
        for inv in inv_data:
                a_val = logger().log_dict[inv]['control_setting']
                init_a = logger().custom_metrics['init_control_settings'][inv]
                t, s = get_translation_and_slope(a_val, init_a)
                t_s_dict[inv] = [t, s]
        return t_s_dict

    ##### 


    inv_control_dict = get_inv_control_dict(adversary_inv)
    inv_control_dict2 = get_inv_control_dict(inverter_list)

    t_s_dict = all_inv_translation_and_slope(adversary_inv)
    t_s_dict2 = all_inv_translation_and_slope(inverter_list)
    
    
    
    def make_control_dataset(inv):
        
        # select control settings for checked inverters
        src = {}
        for i in inv:
            pfix2 = 'inverter_' + str(i)
            pfix = 'adversary_inverter_' + str(i)
            src[i] = [t_s_dict[pfix], t_s_dict2[pfix2]]

        return src

    def make_control_plot(src, inverter_type, geometry_type, title='Control Setting', x_axis_label='Time (s)', y_axis_label=''):
        legend_labels = []
        tag = 0

    
        select_tools = ['box_select', 'lasso_select', 'poly_select', 'tap', 'reset']


        left = figure(plot_width=800, 
                      plot_height=500, 
                      title=title, 
                      x_axis_label=x_axis_label, 
                      y_axis_label=y_axis_label, 
                      toolbar_location='right',
                      tools=select_tools)
   

        for i in src.keys():

            dim = len(src[i][0][0])
            x = list(np.linspace(0, dim, dim))

            for j in range(2): #inverter [1] vs adversary [0]

                for k in range(2): #translation [0] vs slope [1]

                    if j in inverter_type.active and k in geometry_type.active: # plot if selected

                        cds = ColumnDataSource(data=dict(x=x, y0=src[i][j][k])) # format data for plotting

                        p = left.line('x', 'y0',  hover_color="firebrick", source=cds, color=colors[i], 
                                      line_width=3)

                        # hover text based on slope or translation
                        if k == 1:
                            hover = HoverTool(renderers=[p], tooltips=[("slope", "$y")])
                            left.add_tools(hover)

                        else:
                            hover = HoverTool(renderers=[p], tooltips=[("translation", "$y")])
                            left.add_tools(hover)  
                        tag = 1
            if tag == 1:              
                legend_labels.append((i, [p])) # set up legend labels for display outside plot

        if tag == 1:
            legend = Legend(items=legend_labels)
            left.add_layout(legend, 'right') # add legend
        tag = 0


        return left
    
    ######################################
    
#     pos = {}
#     for inv in inv_all:
#         #colors[inv] = next(c)
#         pos[inv] = palette.index(colors[inv])
        
    def make_pq_dataset( node_list, inverter_type, range_start=0, range_end=699):
        
        inv_all = [i for i in log_dict.keys() if 'inverter_' in i]
        df_p_old = {}
        df_q_old = {}
        df_p_new = {}
        df_q_new = {}
        for inv in inv_all:
            df_p_old[inv] = log_dict[inv]['p_out']
            df_p_new[inv] = log_dict[inv]['p_out_new']
            df_q_old[inv] = log_dict[inv]['q_out_new']
            df_q_new[inv] = log_dict[inv]['q_out']

        df_p_old = pd.DataFrame(df_p_old)
        df_p_new = pd.DataFrame(df_p_new)
        df_q_old = pd.DataFrame(df_q_old)
        df_q_new = pd.DataFrame(df_q_new)

        df = [df_p_old, df_q_old, df_p_new, df_q_new]

        for d in df:
            d['total'] = d.sum(axis=1)


        inv_all.append('total')
          
        node_list_new = []
        for j in inverter_type.active:
            if j == 0: #0 is adversary
                lst = ['adversary_inverter_' + node for node in node_list]
                
                node_list_new.append(lst)
            if j == 1:
                lst = ['inverter_' + node for node in node_list]
                
                node_list_new.append(lst)
       
        return [ColumnDataSource(df[i].loc[int(range_start):int(range_end), node_list_new[0]]) for i in range(len(df))]

    def make_pq_plot(src, nodes, inverter_type, title='Real and Reactive Power (PQ)', x_axis_label='Time (s)', y_axis_label=''):
        
        p = figure(plot_width=800, plot_height=400, 
                   title=title,
                   x_range=(src[0].data['index'][0], src[0].data['index'][-1]),
                   x_axis_label=x_axis_label, y_axis_label=y_axis_label, align='end')
        for node in nodes:
            if 1 in inverter_type.active:
                
                #starting_color = itertools.islice(itertools.cycle(palette), pos[inv]+random.randint(1, 6), None)
                glyph1 = p.line(source=src[0], x='index', y='inverter_' + node, legend_label='p_old_' + node[-4:], color=colors[node], 
                               line_width=2)
                glyph2 = p.line(source=src[1], x='index', y='inverter_' +node, legend_label='p_new_' + node[-4:],color=colors[node], 
                               line_width=2)
                glyph3 = p.line(source=src[2], x='index', y='inverter_' +node, legend_label='q_old_' + node[-4:],color=colors[node], 
                                line_width=2)
                glyph4 = p.line(source=src[3], x='index', y='inverter_' +node, legend_label='q_new_' + node[-4:],color=colors[node],line_width=2)
            if 0 in inverter_type.active: # 0 is adversary
                #starting_color = itertools.islice(itertools.cycle(palette), pos[inv]+random.randint(1, 6), None)
                glyph1 = p.line(source=src[0], x='index', y='adversary_inverter_' + node, legend_label='p_old_' + node[-4:], color=colors[node], 
                                 line_width=2)
                glyph2 = p.line(source=src[1], x='index', y='adversary_inverter_' +node, legend_label='p_new_' + node[-4:],color=colors[node],  
                                line_width=2)
                glyph3 = p.line(source=src[2], x='index', y='adversary_inverter_' +node, legend_label='q_old_' + node[-4:],color=colors[node], 
                                line_width=2)
                glyph4 = p.line(source=src[3], x='index', y='adversary_inverter_' +node, legend_label='q_new_' + node[-4:],color=colors[node], 
                                line_width=2)
            hover = HoverTool(renderers=[glyph1, glyph2, glyph3, glyph4], tooltips=[('{}'.format(node), '@{}'.format('$y'))])
            p.add_tools(hover)
            p.legend.location = 'top_left'
       
        return p
    
    ######################################
    
    def update(attr, old, new):
        
        selected_idx = graph.node_renderer.data_source.selected.indices
        graph_node_list= []
        for i in selected_idx:
            graph_node = graph.node_renderer.data_source.data['index'][i]
            
            for col in imbalance_df.columns:
                if str(graph_node) in col:
                    j_clean = col.split('_')[len(col.split('_')) - 1]
                    graph_node_list.append(j_clean)
  
        select_node = list(set(graph_node_list))
       
        range_start = range_select.value[0]
        range_end = range_select.value[1]
        new_subset = make_dataset(imbalance_df, select_node, range_start, range_end)
        src2 = make_u_dataset(u_df, select_node, range_start, range_end)
        
        layout.children[4] = make_plot(new_subset, select_node, inverter_type.active)  #y metric
        layout.children[5] = make_u_plot(src2, select_node)
        
        
        new_subset = make_voltage_dataset(select_node, stats_selection.active)   
        layout.children[2].children[1], legend_it = make_voltage_plot(new_subset, select_node, stats_selection.active)
        
        
        new_subset = make_control_dataset(select_node)
        layout.children[3].children[1] = make_control_plot(new_subset, inverter_type, geometry_type)
        
       
        new_subset = make_pq_dataset(select_node, inverter_type, range_start, range_end)
        for i in range(len(new_subset)):
            src5[i].data.update(new_subset[i].data)
        layout.children[7] = make_pq_plot(src5, select_node, inverter_type)
     

    # adversary/defender button
    inv_LABELS = ["adversary", "defender"]
    inverter_type = CheckboxButtonGroup(labels=inv_LABELS, active=[0], width=150)
    inverter_type.on_change('active', update)
    
    # initial node selection
    node_list_selection = MultiChoice(value=['s701a', 's701b'], options=node_end)
    initial_nodes = node_list_selection.value 

    # adjust the x-axis (time)
    range_select = RangeSlider(start=imbalance_df.index[0], end=imbalance_df.index[imbalance_df.index[len(imbalance_df) - 1]], 
                            value =(imbalance_df.index[0], imbalance_df.index[imbalance_df.index[len(imbalance_df) - 1]]), step=1, title='Time Range (s)')
    range_select.on_change('value', update) 

    # Voltage
    
    LABELS = ["A", "B", "C"]

    stats_selection = CheckboxButtonGroup(labels=LABELS, active=[0, 1], width=100)
    stats_selection.on_change('active', update)

    

    src3 = make_voltage_dataset(initial_nodes, stats_selection.active)
    p3, legend_it = make_voltage_plot(src3, initial_nodes, stats_selection.active)

    
    # initial app
    src = make_dataset(imbalance_df, initial_nodes)
    src2 = make_u_dataset(u_df, initial_nodes)
    p = make_plot(src, initial_nodes, inverter_type.active)
    p2 = make_u_plot(src2, initial_nodes)
    
    
    # translation/slope button
    geometry_LABELS = ["translation", "slope"]
    geometry_type = CheckboxButtonGroup(labels=geometry_LABELS, active=[0], width=150)
    geometry_type.on_change('active', update)

    # retrieve information
    src4 = make_control_dataset(initial_nodes)
    p4 = make_control_plot(src4, inverter_type, geometry_type)   



    src5 = make_pq_dataset( initial_nodes, inverter_type)
    p5 = make_pq_plot(src5, initial_nodes, inverter_type)
   

    ############### network/graph START
    G = networkx.from_pandas_edgelist(x, 'Source', 'Target', 'Weight') 
    G_source = from_networkx(G, networkx.spring_layout, scale=2, center=(0,0))
   

    graph = GraphRenderer()
    node_name = list(G.nodes())
   
    positions = networkx.spring_layout(G, seed=2)
   
    
    def selected_points(attr,old,new):       
        selected_idx = graph.node_renderer.data_source.selected.indices        
        graph.node_renderer.data_source.selected.on_change("indices", update)
         
        
    # Plot object which is updated 
    plot = figure(title = "IEEE 37 Bus Network",
                  tools = "pan, wheel_zoom, box_select, lasso_select, reset, tap", 
                  plot_width = 800, plot_height = 300, 
                  active_drag = "box_select")

    
    # Assign layout for nodes, render graph, and add hover tool
    graph.layout_provider = StaticLayoutProvider(graph_layout=positions)
   # graph.node_renderer.glyph = Ellipse(height=0.05, width=0.05, fill_color="red")#, fill_color="fill_color")
    graph.selection_policy = NodesOnly()
    
    # Visible Labels
    xx, yy = zip(*graph.layout_provider.graph_layout.values())
    node_labels = list(G.nodes())
    source = ColumnDataSource({'xx': xx, 'yy': yy, 'name': [node_labels[i] for i in range(len(xx))]})
    labels = LabelSet(x='xx', y='yy', text='name', 
                      source=source, background_fill_color='white', text_color='blue',
                      text_font_size='10px', background_fill_alpha=.7)
    
    plot.renderers.append(labels)  
    plot.renderers.append(graph)
    
    plot.tools.append(HoverTool(tooltips=[('Bus Node', '@index')]))
    
    graph.node_renderer.data_source.selected.on_change("indices", selected_points)
    
    sub_G = networkx.from_pandas_edgelist(x, 'Source', 'Target', 'Weight')
    sub_graph = from_networkx(sub_G, networkx.spring_layout, scale=2, center=(0,0))     
    graph.edge_renderer.data_source.data = dict(sub_graph.edge_renderer.data_source.data)
    graph.node_renderer.data_source.data = dict(G_source.node_renderer.data_source.data)    
    ################ network/graph end
    

  
    tapx, tapy = make_tap_dataset(['reg1'])
    p_tap = make_tap_plot(tapx, tapy, ['reg1'])
  

    
    layout = Column(Column(plot, range_select), inverter_type, Row(stats_selection, p3), Row(geometry_type, p4),p,  p2, p_tap,  p5)
    
    
    tab = Panel(child = layout, title = 'Y Metric')

    return tab

###############################

In [62]:
#source: https://stackoverflow.com/questions/53603046/get-selected-glyph-from-networkx-graph-in-bokeh
def total_app(doc):

    tab = y_plot()
    tabs = Tabs(tabs = [tab])
   
    doc.add_root(tabs)
    
        
handler = FunctionHandler(total_app)
app = Application(handler)
show(app)

{'s701a': '#1f77b4', 's701b': '#aec7e8', 's701c': '#ff7f0e', 's712c': '#ffbb78', 's713c': '#2ca02c', 's714a': '#98df8a', 's714b': '#d62728', 's718a': '#ff9896', 's720c': '#9467bd', 's722b': '#c5b0d5', 's722c': '#8c564b', 's724b': '#c49c94', 's725b': '#e377c2', 's727c': '#f7b6d2', 's728': '#7f7f7f', 's729a': '#c7c7c7', 's730c': '#bcbd22', 's731b': '#dbdb8d', 's732c': '#17becf', 's733a': '#9edae5', 's734c': '#1f77b4', 's735c': '#aec7e8', 's736b': '#ff7f0e', 's737a': '#ffbb78', 's738a': '#2ca02c', 's740c': '#98df8a', 's741c': '#d62728', 's742a': '#ff9896', 's742b': '#9467bd', 's744a': '#c5b0d5', 'y_mean': '#8c564b', 'y_worst': '#c49c94'}


---
### Old Version App

In [ ]:
def tap_number():
    
    def make_dataset(regname=['reg1']):
        
        x = []
        y = []
        
        for reg in regname:
            x.append(np.arange(len(log_dict[reg]['tap_number'])))
            y.append(log_dict[reg]['tap_number'])
            
        return x, y

    def make_plot(x, y, regname, title='Tap Number Changes', x_axis_label='Time (s)', y_axis_label='Tap Number'):

        p = figure(plot_width=800, plot_height=800, 
                   title=title,
                   x_axis_label=x_axis_label, y_axis_label=y_axis_label)   

        for i in range(len(x)):

            glyph = p.line(x=x[i], y=y[i],legend_label=regname[i])
            hover = HoverTool(renderers=[glyph], tooltips=[("Tap Number", "$y")])
            p.add_tools(hover)

        return p


    x, y = make_dataset(['reg1'])
    p = make_plot(x, y, ['reg1'])


    layout = row(p)
    tab = Panel(child = layout, title = 'Tap Number')
    return tab

In [ ]:
def voltage_plot():

    def generate_voltage_df(phase):
        #phase - base-0

        v_df = pd.DataFrame()

        for val in log_dict['v_metrics'].values():

            for i in val:

                vals = np.array(list(i.values()))
                key = np.array(list(i.keys()))

                temp = pd.DataFrame(data=vals[:, phase], index=key)
                v_df = pd.concat([v_df, temp], axis = 1)

        v_a = v_df.transpose().reset_index().drop(labels='index', axis=1)

        return v_a

    v_a = generate_voltage_df(0)
    v_b = generate_voltage_df(1)
    v_c = generate_voltage_df(2)

    all_voltages = [v_a, v_b, v_c]
    all_node_list = v_c.columns

    c2 = itertools.cycle(palette)
    colors=[]
    for s in range(len(v_c.columns)):
        colors.append(next(c2))
        
    print(colors)

    def make_dataset(node_list, select_stats):
        
        dataset = []
        for i in range(len(select_stats)):
            dataset.append(all_voltages[int(select_stats[i])].loc[:, node_list])
            
        
        return dataset

    def make_plot(src, nodes, phases, title='Voltage by bus', x_axis_label='Time (s)', y_axis_label='Voltage (V)'):
        
        p = figure(plot_width=600, plot_height=600, 
                title=title,
                x_axis_label=x_axis_label, y_axis_label=y_axis_label)    


        p.xaxis.major_label_orientation = "vertical" # alternatively, math.pi/2

        color_dict = {0: 'A', 1:'B', 2:'C'}
        legend_it = []

        for s in range(len(phases)):
            for node in range(len(nodes)):
                subset = src[s].iloc[:, node]

                glyph = p.line(y=list(subset.values), x=list(subset.index), color=colors[node], legend_label=nodes[node])  

                legend_it.append((nodes[node], [glyph]))

                hover = HoverTool(renderers=[glyph], tooltips=[('node {}'.format(nodes[node]), 'phase {}'.format(color_dict[phases[s]])),("Voltage", "$y")])
                p.add_tools(hover)

        return p, legend_it

    def update(attr, old, new):
        select_node = [i for i in node_list_selection.value]
        new_subset = make_dataset(select_node, stats_selection.active)
        #src.update(new_subset)
        layout.children[1], legend_it = make_plot(new_subset, select_node, stats_selection.active)


    node_list_selection = MultiChoice(value=['799', 'sourcebus'], options=np.ndarray.tolist(all_node_list.values))


    node_list_selection.on_change('value', update) #'active update'

    LABELS = ["A", "B", "C"]

    stats_selection = CheckboxButtonGroup(labels=LABELS, active=[0, 1])
    stats_selection.on_change('active', update)

    initial_nodes = [i for i in node_list_selection.value]

    src = make_dataset(initial_nodes, stats_selection.active)
    p, legend_it = make_plot(src, initial_nodes, stats_selection.active)

    selections = Column(node_list_selection, stats_selection)

    controls = Column(selections)


    layout = Column(controls, p)

    tab = Panel(child = layout, title = 'Voltage')

    return tab

#############################

In [ ]:
def y_plot():
    
    node_end = []
    # list of all the nodes that inverters/adversary inverters operate at
    for i in inverter_list:
        temp = i.split('_')[1]
        node_end.append(temp)
    node_end.append('y_mean')
    node_end.append('y_worst')
    
    x = pd.DataFrame({'Source': bus1, 'Target':bus2, 'Weight':np.ones(len(bus2))})

    # store y data
    df = pd.DataFrame()
    for inv in inverter_list:
        df = df.append(pd.Series(log_dict[inv]['y'], name=inv))

    tags = ['y_worst_node', 'y_worst', 'y_mean']
    for tag in tags:
        df = df.append(pd.Series(data=log_dict['y_metrics'][tag], name=tag))
 
    
    y_df = df.transpose()

    # store adversary y data
    adv_y_df = pd.DataFrame()
    for inv in adversary_inv:
        adv_y_df = adv_y_df.append(pd.Series(log_dict[inv]['y'], name=inv))


    adv_y_df = adv_y_df.transpose()
    
    # combine y data
    all_y_df = pd.concat([y_df, adv_y_df], axis=1)

    imbalance_df = all_y_df*100
 
    
    c = itertools.cycle(palette)
    colors = {}
    for node in node_end:
        colors[node] = next(c)


    def make_dataset(dataset, node_list, range_start=0, range_end=650):
             
        updated_dataset = pd.DataFrame()
        
        # select all nodes that are marked
        for node in node_list:
            
            updated_dataset = pd.concat([updated_dataset, dataset.loc[:, [node in s for s in dataset.columns.values]]], axis=1)
        
        # also select y worst and y mean
        updated_dataset = pd.concat([updated_dataset, dataset['y_worst']], axis=1)
        
        updated_dataset = pd.concat([updated_dataset, dataset['y_mean']], axis=1)
        
      

        return ColumnDataSource(updated_dataset.loc[int(range_start):int(range_end), :])

    def make_plot(src, nodes,a_d, title='', x_axis_label='Time (s)', y_axis_label=''):
        

        p = figure(plot_width=600, plot_height=600, 
                 title=title,x_axis_label=x_axis_label, y_axis_label=y_axis_label)

        for node in nodes:
            
            # would have to move this out if using network to represent the nodes
            if node == 'y_worst' or node == 'y_mean':
                    glyph = p.line(source=src, x='index', y=node, legend_label=node, color=colors[node], line_width=2)
                    hover = HoverTool(renderers=[glyph], tooltips=[('{}'.format(node), '$y')])
                    p.add_tools(hover)

            elif 0 in a_d: # attacker/adversary checked
                glyph = p.line(source=src, x='index', y="inverter_"+ node, legend_label=node, color=colors[node], line_width=2)
                hover = HoverTool(renderers=[glyph], tooltips=[('adversary_inverter_{}'.format(node), '$y')])
                p.add_tools(hover)
            elif 1 in a_d: # defender
                glyph = p.line(source=src, x='index', y="inverter_"+ node, legend_label=node, color=colors[node], line_width=2)
                hover = HoverTool(renderers=[glyph], tooltips=[('inverter_{}'.format(node), '$y')])
                p.add_tools(hover)
            
        return p

    def update(attr, old, new):
        
        selected_idx = graph.node_renderer.data_source.selected.indices
        

        graph_node_list= []
        for i in selected_idx:
            graph_node = graph.node_renderer.data_source.data['index'][i]
           
            for col in imbalance_df.columns:
                if str(graph_node) in col:
                    j_clean = col.split('_')[len(col.split('_')) - 1]
                    graph_node_list.append(j_clean)
  
        select_node = list(set(graph_node_list))
        
        range_start = range_select.value[0]
        range_end = range_select.value[1]
        new_subset = make_dataset(imbalance_df, select_node, range_start, range_end)
        #src.data.update(new_subset.data)
        layout.children[1] = make_plot(new_subset, select_node, inverter_type.active)
     

    # adversary/defender button
    inv_LABELS = ["adversary", "defender"]
    inverter_type = CheckboxButtonGroup(labels=inv_LABELS, active=[0])
    inverter_type.on_change('active', update)
    
    # initial node selection
    node_list_selection = MultiChoice(value=['s701a', 's701b'], options=node_end)
    initial_nodes = node_list_selection.value 

    # adjust the x-axis (time)
    range_select = RangeSlider(start=imbalance_df.index[0], end=imbalance_df.index[imbalance_df.index[len(imbalance_df) - 1]], 
                            value =(imbalance_df.index[0], imbalance_df.index[imbalance_df.index[len(imbalance_df) - 1]]), step=1, title='Time Range (s)')
    range_select.on_change('value', update) 

    # initial app
    src = make_dataset(imbalance_df, initial_nodes)
    p = make_plot(src, initial_nodes, inverter_type.active)

    ############### network/graph START
    G = networkx.from_pandas_edgelist(x, 'Source', 'Target', 'Weight') 
    G_source = from_networkx(G, networkx.spring_layout, scale=2, center=(0,0))
   

    graph = GraphRenderer()
    node_name = list(G.nodes())
   
    positions = networkx.spring_layout(G, seed=2)
   
    
    def selected_points(attr,old,new):       
        selected_idx = graph.node_renderer.data_source.selected.indices        
        graph.node_renderer.data_source.selected.on_change("indices", update)
         
        
    # Plot object which is updated 
    plot = figure(title = "IEEE 37 Bus Network", 
                  tools = "pan, wheel_zoom, box_select, lasso_select, reset, tap", 
                  plot_width = 350, plot_height = 500, 
                  active_drag = "box_select")

    
    # Assign layout for nodes, render graph, and add hover tool
    graph.layout_provider = StaticLayoutProvider(graph_layout=positions)
    #graph.node_renderer.glyph = Ellipse(height=0.05, width=0.05, fill_color="red")#, fill_color="fill_color")
    graph.selection_policy = NodesOnly()
    
    # Visible Labels
    xx, yy = zip(*graph.layout_provider.graph_layout.values())
    node_labels = list(G.nodes())
    source = ColumnDataSource({'xx': xx, 'yy': yy, 'name': [node_labels[i] for i in range(len(xx))]})
    labels = LabelSet(x='xx', y='yy', text='name', 
                      source=source, background_fill_color='white', text_color='blue',
                      text_font_size='10px', background_fill_alpha=.7)
    
    plot.renderers.append(labels)  
    plot.renderers.append(graph)
    
    plot.tools.append(HoverTool(tooltips=[('Bus Node', '@index')]))
    
    graph.node_renderer.data_source.selected.on_change("indices", selected_points)
    
    sub_G = networkx.from_pandas_edgelist(x, 'Source', 'Target', 'Weight')
    sub_graph = from_networkx(sub_G, networkx.spring_layout, scale=2, center=(0,0))     
    graph.edge_renderer.data_source.data = dict(sub_graph.edge_renderer.data_source.data)
    graph.node_renderer.data_source.data = dict(G_source.node_renderer.data_source.data)    
    ################ network/graph end
  
    controls = Column(plot, range_select)
    controls = Column(inverter_type, controls)
    
    layout = Row(controls, p)
   
    tab = Panel(child = layout, title = 'Y Metric')

    return tab

###############################

In [ ]:
def u_plot():
    
    imbalance_df = pd.DataFrame(log_dict['u_metrics'])*100
    all_node_list = imbalance_df.columns.to_list()

    imbalance_df['u_std_lower'] = imbalance_df['u_mean'] - imbalance_df['u_std']
    imbalance_df['u_std_upper'] = imbalance_df['u_mean'] + imbalance_df['u_std']

    c = itertools.cycle(palette)
    colors = {}
    for node in all_node_list:
        colors[node] = next(c)

    def make_dataset(dataset, node_list, range_start=0, range_end=650):
        if 'u_std' in node_list:
            node_list.append('u_std_lower')
            node_list.append('u_std_upper')
        return ColumnDataSource(dataset[node_list][int(range_start):int(range_end)])

    def make_plot(src, nodes, title='', x_axis_label='Time (s)', y_axis_label=''):
       
     
        p = figure(plot_width=600, plot_height=600, 
                   title=title,
                   x_range=(src.data['index'][0], src.data['index'][-1]), y_range=(0, 8),
                   x_axis_label=x_axis_label, y_axis_label=y_axis_label)
        for node in nodes:
            if node != 'u_std' and node != 'u_std_lower' and node != 'u_std_upper':

                glyph = p.line(source=src, x='index', y=node, legend_label=node, color=colors[node], line_width=2)
                hover = HoverTool(renderers=[glyph], tooltips=[('imbalance_{}'.format(node), '@{}'.format(node))])#, mode='vline')
                p.add_tools(hover)
            elif node == 'u_std':               
                band = Band(source=src, base='index', lower='u_std_lower', upper='u_std_upper', 
                level='underlay', fill_alpha=0.5, line_width=0.5, fill_color='green')

                p.add_layout(band)
            else:
                pass

        high_box = BoxAnnotation(bottom=6, fill_alpha=0.05, fill_color='red')
        mid_box = BoxAnnotation(bottom=3, top=6, fill_alpha=0.05, fill_color='orange')
        p.add_layout(high_box)
        p.add_layout(mid_box)
        
        return p

    def update(attr, old, new):
        
        select_node = node_list_selection.value
        range_start = range_select.value[0]
        range_end = range_select.value[1]
        new_subset = make_dataset(imbalance_df, select_node, range_start, range_end)
        src.data.update(new_subset.data)
        layout.children[1] = make_plot(src, select_node)

  
    node_list_selection = MultiChoice(value=['701','702'], options=all_node_list)    
    node_list_selection.on_change('value', update)
    range_select = RangeSlider(start=imbalance_df.index.start, end=imbalance_df.index.stop, 
                               value =(imbalance_df.index.start, imbalance_df.index.stop), step=1, title='Time Range (s)')
    range_select.on_change('value', update)

    controls = Column(node_list_selection, range_select)
    
    initial_nodes = node_list_selection.value
    src = make_dataset(imbalance_df, initial_nodes)
    p = make_plot(src, initial_nodes)
    
    layout = row(controls, p)
   
    tab = Panel(child = layout, title = 'U Metric')
    return tab



In [ ]:
def control_setting():

    node_end = []
    # list of all the nodes that inverters/adversary inverters operate at
    for i in inverter_list:
        temp = i.split('_')[1]
        node_end.append(temp)

    ##### Functions
    def get_control_params(k):
        x = np.array(log_dict[k]['control_setting'])
        y = pd.DataFrame(data=x, columns=['control1', 'control2', 'control3', 'control4', 'control5'])
        y = y.transpose()

        return y

    def get_inv_control_dict(inv_data):
        inv_control_dict = {}
        for k in inv_data:
            inv_control_dict[k] = get_control_params(k)
        return inv_control_dict

    def get_translation_and_slope(a_val, init_a):
        points = np.array(a_val)
        slope = points[:, 1] - points[:, 0]
        translation = points[:, 2] - init_a[2]
        return translation, slope

    def all_inv_translation_and_slope(inv_data):
        t_s_dict = {}
        for inv in inv_data:
                a_val = logger().log_dict[inv]['control_setting']
                init_a = logger().custom_metrics['init_control_settings'][inv]
                t, s = get_translation_and_slope(a_val, init_a)
                t_s_dict[inv] = [t, s]
        return t_s_dict

    ##### 

    inv_data = adversary_inv
    inv_data2 = inverter_list

    inv_control_dict = get_inv_control_dict(inv_data)
    inv_control_dict2 = get_inv_control_dict(inv_data2)

    t_s_dict = all_inv_translation_and_slope(inv_data)
    t_s_dict2 = all_inv_translation_and_slope(inv_data2)

    #####

    # set color palette
    c = itertools.cycle(palette)
    colors = {}
    for inv in inv_data:
        colors[inv] = next(c)
    

    def make_dataset(t_s_dict, inv):
       
        # select control settings for checked inverters
        src = {}
        for i in inv:
            pfix2 = 'inverter_' + str(i)
            pfix = 'adversary_inverter_' + str(i)
            src[i] = [t_s_dict[pfix], t_s_dict2[pfix2]]

        return src

    def make_plot(src, inverter_type, geometry_type, title='Control Setting', x_axis_label='Time (s)', y_axis_label=''):
        legend_labels = []
        tag = 0

        
        
        select_tools = ['box_select', 'lasso_select', 'poly_select', 'tap', 'reset']


        left = figure(plot_width=500, 
                      plot_height=700, 
                      title=title, 
                      x_axis_label=x_axis_label, 
                      y_axis_label=y_axis_label, 
                      toolbar_location='below',
                      tools=select_tools)
        


        for i in src.keys():
            color_select = 'adversary_inverter_' + str(i)

            dim = len(src[i][0][0])
            x = list(np.linspace(0, dim, dim))

            for j in range(2): #inverter [1] vs adversary [0]

                for k in range(2): #translation [0] vs slope [1]

                    if j in inverter_type.active and k in geometry_type.active: # plot if selected

                        cds = ColumnDataSource(data=dict(x=x, y0=src[i][j][k])) # format data for plotting

                        p = left.line('x', 'y0',  hover_color="firebrick", source=cds, color=colors[color_select], 
                                      line_width=3)

                        # hover text based on slope or translation
                        if k == 1:
                            hover = HoverTool(renderers=[p], tooltips=[("slope", "$y")])
                            left.add_tools(hover)

                        else:
                            hover = HoverTool(renderers=[p], tooltips=[("translation", "$y")])
                            left.add_tools(hover)  
                        tag = 1
            if tag == 1:              
                legend_labels.append((i, [p])) # set up legend labels for display outside plot

        if tag == 1:
            legend = Legend(items=legend_labels)
            left.add_layout(legend, 'below') # add legend
        tag = 0


        return left

    def update(attr, old, new):
        # checked inverters
        inv_active = [inv_selection.value[i] for i in range(len(inv_selection.value))]  

        # adjust dataset 
        new_subset = make_dataset(t_s_dict, inv_active)

        # update plot
        layout.children[1] = make_plot(new_subset, inverter_type, geometry_type)

    # initalize inverter loads  
    inv_selection = CheckboxGroup(labels=node_end, active=[1, 2])
    inv_selection = MultiChoice(value=['s718a','s720c'], options=node_end)    
    inv_selection.on_change('value', update)

    inv_active = [inv_selection.value[i] for i in range(len(inv_selection.value))]


    # adversary/defender button
    inv_LABELS = ["adversary", "defender"]
    inverter_type = CheckboxButtonGroup(labels=inv_LABELS, active=[0])
    inverter_type.on_change('active', update)

    # translation/slope button
    geometry_LABELS = ["translation", "slope"]
    geometry_type = CheckboxButtonGroup(labels=geometry_LABELS, active=[0])
    geometry_type.on_change('active', update)

    # retrieve information
    src = make_dataset(t_s_dict, inv_active)
    p = make_plot(src, inverter_type, geometry_type)   



    # layout 
    plot_region = Column(p)

    selection_controls = Column(inverter_type, geometry_type)
    controls = Column(selection_controls, inv_selection)

    layout = row(controls, plot_region)

    tab = Panel(child = layout, title = 'VVC/Control Settings')
    return tab



In [ ]:
def pq_plot():
    node_end = []
    # list of all the nodes that inverters/adversary inverters operate at
    for i in inverter_list:
        temp = i.split('_')[1]
        node_end.append(temp)
    node_end.append('y_mean')
    node_end.append('y_worst')

    inv_all = [i for i in log_dict.keys() if 'inverter_' in i]
    df_p_old = {}
    df_q_old = {}
    df_p_new = {}
    df_q_new = {}
    for inv in inv_all:
        df_p_old[inv] = log_dict[inv]['p_out']
        df_p_new[inv] = log_dict[inv]['p_out_new']
        df_q_old[inv] = log_dict[inv]['q_out_new']
        df_q_new[inv] = log_dict[inv]['q_out']

    df_p_old = pd.DataFrame(df_p_old)
    df_p_new = pd.DataFrame(df_p_new)
    df_q_old = pd.DataFrame(df_q_old)
    df_q_new = pd.DataFrame(df_q_new)

    df = [df_p_old, df_q_old, df_p_new, df_q_new]

    for d in df:
        d['total'] = d.sum(axis=1)

    inv_all.append('total')
    c = itertools.cycle(palette)
    colors = {}
    pos = {}
    for inv in inv_all:
        colors[inv] = next(c)
        pos[inv] = palette.index(colors[inv])

    def make_dataset(dataset, node_list, inverter_type, range_start=0, range_end=699):
          
        node_list_new = []
        for j in inverter_type.active:
            if j == 0: #0 is adversary
                lst = ['adversary_inverter_' + node for node in node_list]
                
                node_list_new.append(lst)
            if j == 1:
                lst = ['inverter_' + node for node in node_list]
                
                node_list_new.append(lst)
       
        return [ColumnDataSource(dataset[i].loc[int(range_start):int(range_end), node_list_new[0]]) for i in range(len(dataset))]

    def make_plot(src, nodes, inverter_type, title='', x_axis_label='Time (s)', y_axis_label=''):
        print(nodes)
        p = figure(#plot_width=600, plot_height=600, 
                   title=title,
                   x_range=(src[0].data['index'][0], src[0].data['index'][-1]),
                   x_axis_label=x_axis_label, y_axis_label=y_axis_label)
        for node in nodes:
            if 1 in inverter_type.active:
                
                starting_color = itertools.islice(itertools.cycle(palette), pos[inv]+random.randint(1, 6), None)
                glyph1 = p.line(source=src[0], x='index', y='inverter_' + node, legend_label='p_old_' + node[-4:], color=colors['inverter_' +node], 
                                line_width=2)
                glyph2 = p.line(source=src[1], x='index', y='inverter_' +node, legend_label='p_new_' + node[-4:], color=next(starting_color), 
                                line_width=2)
                glyph3 = p.line(source=src[2], x='index', y='inverter_' +node, legend_label='q_old_' + node[-4:], color=next(starting_color), 
                                line_width=2)
                glyph4 = p.line(source=src[3], x='index', y='inverter_' +node, legend_label='q_new_' + node[-4:], color=next(starting_color), 
                                line_width=2)
            if 0 in inverter_type.active: # 0 is adversary
                starting_color = itertools.islice(itertools.cycle(palette), pos[inv]+random.randint(1, 6), None)
                glyph1 = p.line(source=src[0], x='index', y='adversary_inverter_' + node, legend_label='p_old_' + node[-4:], color=colors['inverter_' +node], 
                                line_width=2)
                glyph2 = p.line(source=src[1], x='index', y='adversary_inverter_' +node, legend_label='p_new_' + node[-4:], color=next(starting_color), 
                                line_width=2)
                glyph3 = p.line(source=src[2], x='index', y='adversary_inverter_' +node, legend_label='q_old_' + node[-4:], color=next(starting_color), 
                                line_width=2)
                glyph4 = p.line(source=src[3], x='index', y='adversary_inverter_' +node, legend_label='q_new_' + node[-4:], color=next(starting_color), 
                                line_width=2)
            hover = HoverTool(renderers=[glyph1, glyph2, glyph3, glyph4], tooltips=[('{}'.format(node), '@{}'.format(node))])
            p.add_tools(hover)
            p.legend.location = 'top_left'
       
        return p

    def update(attr, old, new):
        
        select_node = [i for i in inv_list_selection.value]
        range_start = range_select.value[0]
        range_end = range_select.value[1]
        new_subset = make_dataset(df, select_node, inv_type, range_start, range_end)
        for i in range(len(new_subset)):
            src[i].data.update(new_subset[i].data)
        layout.children[1] = make_plot(src, select_node, inv_type)

    #inv_list_selection = CheckboxGroup(labels=inv_all, active=[1, 2])
    inv_list_selection = CheckboxGroup(labels=node_end, active=[1, 2])
    inv_list_selection = MultiChoice(options=node_end, value=['s701a', 's701c'])
    inv_list_selection.on_change('value', update)
    
    # adversary/defender button
    inv_labels = ["adversary", "defender"]
    inv_type = CheckboxButtonGroup(labels=inv_labels, active=[0])
    inv_type.on_change('active', update)
    
    
    range_select = RangeSlider(start=df[0].index.start, end=df[0].index.stop, 
                               value =(df[0].index.start, df[0].index.stop), step=1, title='Time Range (s)')
    range_select.on_change('value', update)

    controls = Column(inv_list_selection, range_select)
    controls = Column(inv_type, controls)
    initial_nodes = [i for i in inv_list_selection.value]
    src = make_dataset(df, initial_nodes, inv_type)
    p = make_plot(src, initial_nodes, inv_type)

    layout = Column(controls, p)

    tab = Panel(child = layout, title = 'PQ')
    return tab


In [ ]:
#source: https://stackoverflow.com/questions/53603046/get-selected-glyph-from-networkx-graph-in-bokeh
def total_app(doc):

    tab = y_plot()

    tab2 = voltage_plot()
    
    tab3 = tap_number()
    
    tab4 = u_plot()
    
    tab5 = control_setting()
    
    tab6 = pq_plot()


    tabs = Tabs(tabs = [tab6, tab5, tab2, tab3, tab4, tab])
   
    doc.add_root(tabs)
    
        
handler = FunctionHandler(total_app)
app = Application(handler)


In [ ]:
show(app)

---

inverter specific power/voltage

In [ ]:
s701a = pd.DataFrame(index=log_dict['s701a'].keys(), data=log_dict['s701a'].values())
s701a.head()

In [ ]:
log_dict['inverter_s701a'].keys()

---

network

In [ ]:
# initalize network df with data from log_dic['network']
network_df = pd.DataFrame(index=log_dict['network'].keys(), data=log_dict['network'].values())

# parse substation power and split into re and im parts
substationpower = np.array([0, 0])
for n in network_df.loc['substation_power', :]:
    substationpower = np.vstack((substationpower, [n[0], n[1]]))

# same for loss
loss = np.array([0, 0])
for n in network_df.loc['loss', :]:
    loss = np.vstack((loss, [n[0], n[1]]))
loss = loss[1:, :]

# move from np array to df 
loss_df = pd.DataFrame(data = loss, columns=['re_loss', 'im_loss'])
subpower_df = pd.DataFrame(data = substationpower, columns=['re_sspower', 'im_sspower'])

# update df to have loss/substation power separated out 
network_df = network_df.append(loss_df.transpose()).append(subpower_df.transpose()).drop('substation_power').drop('loss')
network_df

---

component reward

In [ ]:
print(np.array(list(log_dict['component_reward'].values())).shape)
component_df = pd.DataFrame(index=log_dict['component_reward'].keys(), data=log_dict['component_reward'].values())
component_df.head()

---

current data

In [ ]:
a = np.array(list(log_dict['current'].values()))
#a.shape = (lines, time_steps, phases)

current_a_df = pd.DataFrame(index=log_dict['current'].keys(), data=a[:,:,0])


current_b_df = pd.DataFrame(index=log_dict['current'].keys(), data=a[:,:,1])


current_c_df = pd.DataFrame(index=log_dict['current'].keys(), data=a[:,:,2])

current_b_df

In [ ]:
%matplotlib inline
plt.figure()
for i in range(0, 5):
    current_a_df.iloc[i, :].plot()
plt.title('current along phase a')
plt.legend(['line 1', 'line 2', 'line3', 'line4', 'line5'])

In [ ]:
all_line_list = (current_a_df.index)

def current_plot(doc):

    c2 = itertools.cycle(palette)
    colors={}
    for s in list(all_line_list):
        colors[s] = next(c2)
  
    complete_data = [current_a_df, current_b_df, current_c_df]

    def make_dataset(dataset, node_list, select_stats):
        phased_data = []
        
        for i in select_stats:
             
            phased_data.append(dataset[i].iloc[node_list])
  
        return phased_data

    def make_plot(src, nodes, stats, title='Current along a line', x_axis_label='Time (s)', y_axis_label='Current (A)'):
        p = figure(plot_width=600, plot_height=600, title=title,x_axis_label=x_axis_label, y_axis_label=y_axis_label)    

        
        for phase in range(len(src)):
      
            for r in range(len(src[phase].index)):
               
                c = colors[src[phase].index[r]]
                p.circle(y=src[phase].iloc[r, :], x=src[phase].columns, fill_color = c, line_color=c)

            p.xaxis.major_label_orientation = "vertical" # alternatively, math.pi/2

        p.legend.title = "Current Measurements"
        p.legend.location = "bottom_right"

        return p

    def update(attr, old, new):
        select_node = [i for i in line_list_selection.active]
        select_stats = [stats_selection.labels[i] for i in stats_selection.active]
        print('active phases', stats_selection.active) #indices of the labels
   
        new_subset = make_dataset(complete_data, select_node, stats_selection.active)
        #src.update(new_subset)
       
        layout.children[1] = make_plot(new_subset, select_node, select_stats)
              
  
    line_list_selection = CheckboxGroup(labels=np.ndarray.tolist(all_line_list.values), active=[1, 2])
    line_list_selection.on_change('active', update) #'active update'
   

    LABELS = ["current_a", "current_b", "current_c"]

    stats_selection = CheckboxButtonGroup(labels=LABELS, active=[0, 1])
    stats_selection.on_change('active', update)
   
    selections = Column(line_list_selection, stats_selection)
    controls = Column(selections)
    initial_nodes = [i for i in line_list_selection.active]
    initial_stats = [stats_selection.labels[i] for i in stats_selection.active]
      
    src = make_dataset(complete_data, initial_nodes, stats_selection.active )
    p = make_plot(src, initial_nodes, stats_selection.active)

    layout = row(controls, p)
    doc.add_root(layout)
    
# Set up an application
handler = FunctionHandler(current_plot)
app = Application(handler)


In [ ]:
show(app)

---

inverter data

control settings: 

In [ ]:
# create df of adversary inverter parameters
adv701b = pd.DataFrame(index=log_dict['adversary_inverter_s701b'].keys(), data=log_dict['adversary_inverter_s701b'].values())

# separate out vvc parameters
x = np.array(log_dict['adversary_inverter_s701b']['control_setting'])
y = pd.DataFrame(data=x, columns=['control1', 'control2', 'control3', 'control4', 'control5'])
y = y.transpose()

# update vvc in dataframe
adv701b = adv701b.drop('control_setting').append(y)

adv701b

In [ ]:
def get_control_params(k):
    x = np.array(log_dict[k]['control_setting'])
    y = pd.DataFrame(data=x, columns=['control1', 'control2', 'control3', 'control4', 'control5'])
    y = y.transpose()
    
    return y

In [ ]:
inv_control_dict = {}
for k in adversary_inv:#inverter_list:
    inv_control_dict[k] = get_control_params(k)

In [ ]:
inv_control_dict

--- 